<a href="https://colab.research.google.com/github/NahariImane/NLP_TP/blob/main/TP3_NLP_Nahari_Imane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk gensim scikit-learn pandas numpy

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec


In [7]:
# Charger le dataset
df = pd.read_csv('movie_review.csv')

In [9]:
# Pré-processing des données textuelles
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    # Supprimer les stop words
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

In [11]:
df['text'] = df['text'].apply(preprocess_text)

In [13]:
# Entraînement du modèle Word2Vec
tokenized_reviews = [word_tokenize(review) for review in df['text']]
model_w2v = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)


In [14]:
# Vectorisation des reviews de films
def average_word2vec(review):
    tokens = word_tokenize(review)
    vectors = [model_w2v.wv[word] for word in tokens if word in model_w2v.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)  # Vecteur nul si aucun mot n'est présent dans le modèle

df['Vectorized_Review'] = df['text'].apply(average_word2vec)

In [16]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(np.vstack(df['Vectorized_Review']), df['tag'], test_size=0.2, random_state=42)


In [17]:
# Construction d'un classificateur (utilisation de Logistic Regression ici)
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [18]:
# Prédiction sur l'ensemble de test
y_pred = classifier.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')


In [19]:

# Affichage des métriques d'évaluation
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.5710754017305315
Precision: 0.5730518695133472
Recall: 0.5710754017305315
F1 Score: 0.5652602905365949
